In [2]:
#imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [5]:
# create Fully Connected Neural Network

class NN(nn.Module):
    def __init__(self , input_size , num_classes):
        super(NN , self).__init__()
        self.fc1 = nn.Linear(input_size , 50)
        self.fc2 = nn.Linear(50 , num_classes)
    
    def forward(self , x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [10]:
# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
# Hyperparameters 

input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 1

In [16]:
# Load Data
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset , batch_size=batch_size , shuffle=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset , batch_size=batch_size , shuffle=True)

In [17]:
# initialize the network
model = NN(input_size , num_classes).to(device)

In [18]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters() , lr=learning_rate)

In [20]:
# Train the netwrork
for epoch in range(num_epoch):
    for batch_idx , (data , target) in enumerate(train_loader):
        # get data to cuda if possible
        data = data.to(device)
        target = target.to(device)

        # get to correct shape
        data = data.reshape(data.shape[0] , -1)

        #forward
        scores = model(data)
        loss = criterion(scores , target)

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step()

In [27]:
# check the accuracy 

def check_accuracy(loader , model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x , y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0] , -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples) * 100:.2f} ')
    
    model.train()    


In [28]:
check_accuracy(train_loader , model)
check_accuracy(test_loader , model)

Got 56222 / 60000 with accuracy 93.70 
Got 9350 / 10000 with accuracy 93.50 
